In [10]:
import pandas as pd

In [36]:
import time
import requests
from bs4 import BeautifulSoup

def _get_deps(row):
    result = get_dependents(row["full_name"], 10, 10)
    row['dependents'] =  [t[0] for t in result]
    row['dependent_stars'] =  [t[1] for t in result]
    return row

    
def get_dependents(repo, min_stars_cnt, result_cnt):
    url = 'https://github.com/{}/network/dependents'.format(repo)
    nextExists = True

    result = []
    while nextExists and len(result) < result_cnt:
        # uncomment the line below to see progress.
        print("url: " + url + "  " + "cnt: " + str(len(result)))
        r = requests.get(url)
        soup = BeautifulSoup(r.content, "html.parser")
    
        tmp = [
            (
                 "{}/{}".format(
                    t.find('a', {"data-repository-hovercards-enabled":""}).text,
                    t.find('a', {"data-hovercard-type":"repository"}).text
                 ),
                 int(t.find("svg", {"class": "octicon-star"}).parent.text.strip().replace(',', ''))
            )
            for t in soup.findAll("div", {"class": "Box-row"})
        ]
        tmp = list(filter(lambda repo: repo[1] > min_stars_cnt, tmp))
        result = result + tmp
        nextExists = False
        try:
            container = soup.find("div", {"class":"paginate-container"})
            if container is not None:
                for u in container.findAll('a'):
                    if u.text == "Next":
                        nextExists = True
                        url = u["href"]
        except Exception as e:
            print(e)
            print("waiting for 10 seconds...")
            time.sleep(10)
            nextExists = True
    return result 

In [ ]:
repo = "huggingface/transformers"

for r in get_dependents(repo, 10, 10):
  print(r["name"] + ", " + str(r["stars"]))

In [29]:
df_impt = pd.read_csv("important-ai-projects.csv").sort_values("contributors", ascending=False).head(500)

In [ ]:
df_dep = df_impt.apply(lambda r: _get_deps(r) , axis=1)

df_out = df_dep.explode(column=["dependents", "dependent_stars"], ignore_index=True)
df_out.to_csv("top500-deps.csv", index=False)
df_out

In [48]:
df_out = pd.read_csv("top500-deps.csv")
df_out = df_out.query("dependents == dependents").rename(columns={"full_name": "dest_repo", "dependents": "src_repo", "dependent_stars": "weight"})
df_out = df_out.astype({"weight": "int32"})
df_out.to_csv("togephi.csv", columns=["src_repo", "dest_repo", "weight"], index=False)

In [ ]:
url = 'https://github.com/avelino/awesome-go/tree/main'
response = requests.get(url)
tree = html.fromstring(response.content)"

cats = tree.xpath('//p[@dir="auto"]/preceding-sibling::h2')
dikts = []
for cat in cats:
    cat_name = cat.xpath('./text()')[0]
    lis = cat.xpath('./following-sibling::ul[1]/li')
    for li in lis:
        desc = li.xpath('./text()')
        if type(desc) != 'list' and len(desc) != 1: continue
        link = li.xpath('./a/@href')
        if type(link) != 'list' and len(link) != 1: continue
        dikts.append({
            "category": cat_name,
            "full_name": "/".join(link[0].rsplit('/', 2)[1:]),
            "link": link[0],
            "description": desc[0][3:],
        })
df = pd.DataFrame(dikts)
